In [ ]:
# BLOQUE 1: Introducción y objetivos

# Este proyecto simula una base de datos financiera de un banco ficticio
# con datos sobre clientes, transacciones, cuentas y créditos.

# Objetivo: demostrar cómo extraer datos financieros usando SQL y Python,
# analizarlos de forma profesional y presentar conclusiones accionables.

# Librerías necesarias
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings

warnings.filterwarnings("ignore")
sns.set(style="whitegrid")


In [ ]:
# BLOQUE 2: Generación de datos simulados

# Semilla para reproducibilidad
np.random.seed(42)

# 1. Tabla de clientes
n_clients = 500
clientes = pd.DataFrame({
    "cliente_id": range(1, n_clients + 1),
    "nombre": [f"Cliente_{i}" for i in range(1, n_clients + 1)],
    "edad": np.random.randint(18, 75, size=n_clients),
    "genero": np.random.choice(["M", "F"], size=n_clients),
    "ciudad": np.random.choice(["Madrid", "Barcelona", "Valencia", "Sevilla", "Bilbao"], size=n_clients),
    "fecha_registro": pd.to_datetime(np.random.randint(2015, 2023, size=n_clients), format='%Y') + pd.to_timedelta(np.random.randint(0, 365, size=n_clients), unit='D')
})

# 2. Tabla de cuentas
cuentas = pd.DataFrame({
    "cuenta_id": range(1, n_clients + 1),
    "cliente_id": clientes["cliente_id"],
    "tipo_cuenta": np.random.choice(["Corriente", "Ahorros", "Inversión"], size=n_clients, p=[0.5, 0.3, 0.2]),
    "saldo": np.round(np.random.normal(5000, 3000, size=n_clients), 2),
    "fecha_apertura": clientes["fecha_registro"]
})

# 3. Tabla de transacciones
n_transacciones = 5000
transacciones = pd.DataFrame({
    "transaccion_id": range(1, n_transacciones + 1),
    "cuenta_id": np.random.choice(cuentas["cuenta_id"], size=n_transacciones),
    "fecha": pd.to_datetime("2023-01-01") + pd.to_timedelta(np.random.randint(0, 365, size=n_transacciones), unit='D'),
    "monto": np.round(np.random.normal(200, 150, size=n_transacciones), 2),
    "tipo": np.random.choice(["Ingreso", "Gasto"], size=n_transacciones, p=[0.4, 0.6])
})

# Ajustar signos
transacciones.loc[transacciones["tipo"] == "Gasto", "monto"] *= -1

# 4. Tabla de créditos
creditos = pd.DataFrame({
    "credito_id": range(1, 300 + 1),
    "cliente_id": np.random.choice(clientes["cliente_id"], size=300, replace=False),
    "monto_credito": np.random.randint(2000, 30000, size=300),
    "interes_anual": np.round(np.random.uniform(2.5, 10.0, size=300), 2),
    "estado": np.random.choice(["Activo", "Pagado", "Incobrable"], size=300, p=[0.6, 0.3, 0.1]),
    "fecha_otorgamiento": pd.to_datetime("2022-01-01") + pd.to_timedelta(np.random.randint(0, 700, size=300), unit='D')
})

# Mostrar ejemplo
clientes.head(3), cuentas.head(3), transacciones.head(3), creditos.head(3)


In [ ]:
# BLOQUE 3: Crear base de datos SQLite e insertar datos

# Crear una base de datos SQLite en disco (se guarda como archivo .db)
conexion = sqlite3.connect("banco_analisis.db")
cursor = conexion.cursor()

# Guardar cada DataFrame como tabla en SQL
clientes.to_sql("clientes", conexion, if_exists="replace", index=False)
cuentas.to_sql("cuentas", conexion, if_exists="replace", index=False)
transacciones.to_sql("transacciones", conexion, if_exists="replace", index=False)
creditos.to_sql("creditos", conexion, if_exists="replace", index=False)

# Verificamos que las tablas fueron creadas correctamente
consulta = "SELECT name FROM sqlite_master WHERE type='table';"
cursor.execute(consulta)
tablas = cursor.fetchall()
print("Tablas creadas en la base de datos:", tablas)


In [ ]:
# BLOQUE 4: Consultas SQL básicas

# ¿Cuántos clientes hay en total?
consulta = "SELECT COUNT(*) AS total_clientes FROM clientes;"
total_clientes = pd.read_sql_query(consulta, conexion)
print("Total de clientes:", total_clientes["total_clientes"].values[0])

# ¿Saldo promedio por tipo de cuenta?
consulta = """
SELECT tipo_cuenta, ROUND(AVG(saldo), 2) AS saldo_promedio
FROM cuentas
GROUP BY tipo_cuenta
ORDER BY saldo_promedio DESC;
"""
pd.read_sql_query(consulta, conexion)


In [ ]:
# Total de transacciones por tipo
consulta = """
SELECT tipo, COUNT(*) AS cantidad, ROUND(SUM(monto), 2) AS total_monto
FROM transacciones
GROUP BY tipo;
"""
pd.read_sql_query(consulta, conexion)

# Créditos activos vs pagados
consulta = """
SELECT estado, COUNT(*) AS cantidad_creditos, ROUND(AVG(monto_credito), 2) AS promedio_credito
FROM creditos
GROUP BY estado;
"""
pd.read_sql_query(consulta, conexion)


In [ ]:
# BLOQUE 5: EDA - Distribución de saldos
plt.figure(figsize=(10, 5))
sns.histplot(cuentas["saldo"], bins=30, kde=True, color="teal")
plt.title("Distribución de saldos de cuentas")
plt.xlabel("Saldo (€)")
plt.ylabel("Frecuencia")
plt.show()


In [ ]:
# Saldos promedio por ciudad
consulta = """
SELECT c.ciudad, ROUND(AVG(cu.saldo), 2) AS saldo_promedio
FROM clientes c
JOIN cuentas cu ON c.cliente_id = cu.cliente_id
GROUP BY c.ciudad
ORDER BY saldo_promedio DESC;
"""
df_saldos = pd.read_sql_query(consulta, conexion)
px.bar(df_saldos, x="ciudad", y="saldo_promedio", title="Saldo promedio por ciudad", color="saldo_promedio")


In [ ]:
# BLOQUE 6: Indicadores Financieros Clave

# 1. Ingreso Neto del banco
consulta = """
SELECT 
    SUM(CASE WHEN tipo = 'Ingreso' THEN monto ELSE 0 END) AS total_ingresos,
    ABS(SUM(CASE WHEN tipo = 'Gasto' THEN monto ELSE 0 END)) AS total_gastos,
    SUM(monto) AS ingreso_neto
FROM transacciones;
"""
kpis = pd.read_sql_query(consulta, conexion)
kpis


In [ ]:
# Ratio de morosidad (créditos incobrables / total créditos)
consulta = """
SELECT 
    ROUND(SUM(CASE WHEN estado = 'Incobrable' THEN 1 ELSE 0 END) * 1.0 / COUNT(*), 4) AS ratio_morosidad
FROM creditos;
"""
morosidad = pd.read_sql_query(consulta, conexion)
morosidad


In [ ]:
# Rentabilidad Promedio de los créditos activos
consulta = """
SELECT ROUND(AVG(interes_anual), 2) AS rentabilidad_promedio
FROM creditos
WHERE estado = 'Activo';
"""
rentabilidad = pd.read_sql_query(consulta, conexion)
rentabilidad


In [ ]:
# Evolución de transacciones en el tiempo
consulta = """
SELECT strftime('%Y-%m', fecha) AS mes, 
       SUM(CASE WHEN tipo = 'Ingreso' THEN monto ELSE 0 END) AS total_ingresos,
       ABS(SUM(CASE WHEN tipo = 'Gasto' THEN monto ELSE 0 END)) AS total_gastos
FROM transacciones
GROUP BY mes
ORDER BY mes;
"""
evolucion = pd.read_sql_query(consulta, conexion)
fig = px.line(evolucion, x='mes', y=['total_ingresos', 'total_gastos'], 
              title="Evolución mensual de ingresos y gastos",
              labels={"value": "Euros", "mes": "Mes"}, markers=True)
fig.show()


In [ ]:
# Distribución de estado de créditos
consulta = """
SELECT estado, COUNT(*) AS cantidad
FROM creditos
GROUP BY estado;
"""
creditos_estado = pd.read_sql_query(consulta, conexion)
fig = px.pie(creditos_estado, names='estado', values='cantidad', title='Distribución de estados de crédito', hole=0.4)
fig.show()


In [ ]:
# Relación entre edad del cliente y saldo promedio
consulta = """
SELECT c.edad, ROUND(AVG(cu.saldo), 2) AS saldo_promedio
FROM clientes c
JOIN cuentas cu ON c.cliente_id = cu.cliente_id
GROUP BY c.edad
ORDER BY c.edad;
"""
edad_saldo = pd.read_sql_query(consulta, conexion)
fig = px.scatter(edad_saldo, x="edad", y="saldo_promedio", trendline="ols",
                 title="Relación entre edad y saldo promedio")
fig.show()


In [ ]:
# Rentabilidad y morosidad por ciudad
consulta = """
SELECT 
    c.ciudad,
    COUNT(DISTINCT cl.cliente_id) AS clientes,
    ROUND(AVG(cu.saldo), 2) AS saldo_promedio,
    COUNT(DISTINCT cr.credito_id) AS cantidad_creditos,
    ROUND(AVG(cr.monto_credito), 2) AS promedio_credito,
    SUM(CASE WHEN cr.estado = 'Incobrable' THEN 1 ELSE 0 END) AS creditos_incobrables
FROM clientes cl
LEFT JOIN cuentas cu ON cl.cliente_id = cu.cliente_id
LEFT JOIN creditos cr ON cl.cliente_id = cr.cliente_id
LEFT JOIN clientes c ON c.cliente_id = cl.cliente_id
GROUP BY c.ciudad;
"""
ciudad_stats = pd.read_sql_query(consulta, conexion)
ciudad_stats["morosidad_%"] = round((ciudad_stats["creditos_incobrables"] / ciudad_stats["cantidad_creditos"]) * 100, 2)
ciudad_stats.fillna(0, inplace=True)
ciudad_stats.sort_values("morosidad_%", ascending=False)


In [ ]:
# Antigüedad del cliente en años y saldo promedio
consulta = """
SELECT 
    cl.cliente_id,
    ROUND((julianday('now') - julianday(cl.fecha_registro)) / 365.25, 1) AS antiguedad_anos,
    cu.saldo
FROM clientes cl
JOIN cuentas cu ON cl.cliente_id = cu.cliente_id;
"""
vida_cliente = pd.read_sql_query(consulta, conexion)
fig = px.scatter(vida_cliente, x="antiguedad_anos", y="saldo", trendline="ols",
                 title="Relación entre antigüedad del cliente y saldo")
fig.show()


In [ ]:
# Interés medio según estado del crédito
consulta = """
SELECT estado, COUNT(*) AS cantidad,
       ROUND(AVG(interes_anual), 2) AS interes_promedio,
       ROUND(AVG(monto_credito), 2) AS credito_promedio
FROM creditos
GROUP BY estado;
"""
pd.read_sql_query(consulta, conexion)


In [ ]:
# Transacciones por cuenta
consulta = """
SELECT t.cuenta_id, COUNT(*) AS num_transacciones, 
       ROUND(SUM(t.monto), 2) AS total_monto,
       cu.saldo, cu.tipo_cuenta
FROM transacciones t
JOIN cuentas cu ON t.cuenta_id = cu.cuenta_id
GROUP BY t.cuenta_id;
"""
transacciones_cuenta = pd.read_sql_query(consulta, conexion)

# Relación entre número de transacciones y saldo
fig = px.scatter(transacciones_cuenta, x="num_transacciones", y="saldo", color="tipo_cuenta",
                 title="Relación entre número de transacciones y saldo")
fig.show()


In [ ]:
# ¿Cuánto representa el top 10 de clientes en el total de saldos?
consulta = """
SELECT cliente_id, SUM(saldo) AS saldo_total
FROM cuentas
GROUP BY cliente_id
ORDER BY saldo_total DESC
LIMIT 50;
"""
top_clientes = pd.read_sql_query(consulta, conexion)
total_saldo = pd.read_sql_query("SELECT SUM(saldo) AS total_saldo FROM cuentas", conexion).iloc[0, 0]
top_10_pct = round(top_clientes.head(10)["saldo_total"].sum() / total_saldo * 100, 2)
top_50_pct = round(top_clientes["saldo_total"].sum() / total_saldo * 100, 2)

print(f"🔝 El top 10 de clientes concentra el {top_10_pct}% del saldo total del banco.")
print(f"🔝 El top 50 de clientes concentra el {top_50_pct}% del saldo total del banco.")


In [ ]:
# Crear un archivo Excel con múltiples hojas
ruta_excel = "analisis_financiero_banco.xlsx"

with pd.ExcelWriter(ruta_excel, engine='xlsxwriter') as writer:
    # Datos base
    clientes.to_excel(writer, sheet_name="Clientes", index=False)
    cuentas.to_excel(writer, sheet_name="Cuentas", index=False)
    transacciones.to_excel(writer, sheet_name="Transacciones", index=False)
    creditos.to_excel(writer, sheet_name="Creditos", index=False)
    
    # Métricas financieras
    kpis.to_excel(writer, sheet_name="Resumen KPIs", index=False)
    morosidad.to_excel(writer, sheet_name="Morosidad", index=False)
    rentabilidad.to_excel(writer, sheet_name="Rentabilidad", index=False)
    ciudad_stats.to_excel(writer, sheet_name="Riesgo por Ciudad", index=False)
    vida_cliente.to_excel(writer, sheet_name="Ciclo Cliente", index=False)
    transacciones_cuenta.to_excel(writer, sheet_name="Comportamiento Transaccional", index=False)
    top_clientes.to_excel(writer, sheet_name="Top Clientes", index=False)

print("✅ Excel generado exitosamente.")
